In [53]:
## Importing Libraries ##
import pandas as pd
import numpy as np
import seaborn as sns
import xlrd
import openpyxl
from timeit import default_timer as timer
from datetime import datetime
from sklearn import *
import copy

In [54]:
## Importing libraries ##
df = pd.read_csv('cleaning_water_height_dataset.csv')
event_data = pd.read_csv('cleaned_event_dataset.csv')
accum_data = pd.read_csv('cleaned_accumulation_dataset.csv')

## Definting dataframe ## 
df_concat = pd.DataFrame()

## Setting focused variable ##
target = 'Billinudgel (Marshalls Creek)'
tar_days = 4

In [56]:
## Processing data ##

## Processing rain data ##
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index('Unnamed: 0')

## Processing rain data ##
event_data['Unnamed: 0'] = pd.to_datetime(event_data['Unnamed: 0'])
event_data = event_data.set_index('Unnamed: 0')

## Processing Guage data ##
accum_data.OBS_TIME_LOC = pd.to_datetime(accum_data['OBS_TIME_LOC'])
accum_data = accum_data.set_index('OBS_TIME_LOC')

In [57]:
## Feature manipulation ##
MA, SMA, WSMA = [], [], []

for i in range(0,len(df)):
    MA.append((df[str(target)][i-1-tar_days]+df[str(target)][i-tar_days])/2 + 
    (df[str(target)][i-tar_days] - df[str(target)][i-1-tar_days]))

for i in range(0,len(df)):
    SMA.append((((df[str(target)][i-1-tar_days]+df[str(target)][i-tar_days])/2 + 
    (df[str(target)][i-tar_days] - df[str(target)][i-1-tar_days]))*2  +
    ((df[str(target)][i-2-tar_days]+df[str(target)][i-tar_days])/2 + 
    (df[str(target)][i-tar_days] - df[str(target)][i-2-tar_days])))/3)
    
for i in range(0,len(df)):
    WSMA.append((MA[i-tar_days]*0.1) + (df[str(target)][i-tar_days]*0.9))

## Definting variables ##
df['moving_average'], df['smooth_moving_average'], df['weakly_smooth_moving_average'] = MA, SMA, WSMA 

In [58]:
## Merging datasets ##
merged = pd.concat([df, event_data], axis=1)
df = merged.dropna()

## Merging dataset guage ##
merged  = pd.concat([accum_data,df],axis=1)
merged  = merged.dropna()

In [59]:
## Feature manipulation ## 
merged ['maxdiff'] = merged.diff().max(axis=1)

## Retrieving future and previous target results ##
merged ['future_pred'] = merged [str(target)].shift(-tar_days)
merged ['prev'] = merged [str(target)].shift(5)
merged ['prevWeek'] = merged [str(target)].shift(14) 
merged  = merged.dropna()

In [61]:
## Importing to merge files ##
merged.to_csv('mergedFiles.csv')